In [ ]:
''' 1. '''
import networkx as nx
import random
import copy
import time
import matplotlib.pyplot as plt
import sys
from collections import defaultdict
%matplotlib inline

def progressBar(value, endvalue, bar_length=20):

        percent = float(value) / endvalue
        arrow = '-' * int(round(percent * bar_length)-1) + '>'
        spaces = ' ' * (bar_length - len(arrow))

        sys.stdout.write("\rCalculating: [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
        sys.stdout.flush()

        
def kawai_draw(G, infected=[]) :
    colors = []
    for node in G :
        if node in infected :
            colors.append('red')
        else :
            colors.append('green')
    nx.draw(G, layout=nx.kamada_kawai_layout(G), node_color=colors, node_size = 30, alpha = 0.7)
    plt.show()
    
G = nx.scale_free_graph(1000)
G = G.to_undirected()

#G = nx.watts_strogatz_graph(1000, 10, 0.05)

In [ ]:
''' 2.
    For OLDER version '''
def generate_egos(G, node, max_radius):
    resulting_G = []
    
    G_ego = nx.ego_graph(G, node, radius=1)
    if len(G_ego) > 1 :
        resulting_G.append(G_ego)
        G_ego_previous = G_ego

        for i in range(2, max_radius+1) :
            G_ego = nx.ego_graph(G, node, radius=i)
            G_ego_toremove = copy.deepcopy(G_ego)
            G_ego_toremove.remove_nodes_from(G_ego_previous)
            G_ego_previous = G_ego

            if len(G_ego_toremove) > 0 :
                resulting_G.append(G_ego_toremove)
            else :
                break

        resulting_G[0].remove_node(node)
    return resulting_G

def generate_egos_allnodes(G, max_radius) :
    result = []
    
    nodecount = 1
    for node in G :
        progressBar(nodecount, len(G), 50)
        result.append(generate_egos(G, node, max_radius))
        nodecount +=1
        
    return result

radius_split = generate_egos_allnodes(G, 10)

In [ ]:
''' 3.
    For OLDER version '''
def infected_percentage(G, infected) :    
    count = 0
    
    for node in G :
        if node in infected :
            count += 1
        
    return count / len(G)

def infected_average_by_radius(G, Gs_list, radius, infected) :
    node_count = []
    percentage_sum = []
    for _ in range(0, radius) :
        node_count.append(0)
        percentage_sum.append(0)
            
    current_node = 0
    for node in Gs_list :
        if current_node in infected :
            current_radius = 0
            for item in node :
                percentage_sum[current_radius] += infected_percentage(item, infected)
                node_count[current_radius] += 1
                current_radius += 1
        current_node += 1
        
    average_by_radius = []
    for i in range(0, radius) :
        if node_count[i] > 0 :
            average_by_radius.append(percentage_sum[i] / node_count[i])
        
    return average_by_radius


In [ ]:
''' 6.
    SCROLL DOWN NOW, since this is
    For NEWER version '''
import networkx as nx
import copy
import time
start = time.time()

spl = nx.all_pairs_shortest_path_length(G)

def change_ds(spl_gen, num_nodes, max_radius):
    ''' Transforms 'spl_gen' into the accepted datastructure
        by the 'infected_average_by_separation_degree' method
    '''
    result = []
    for n in range(num_nodes):
        di_tmp = {}
        for entry in spl_gen:  
            for radius in range(1, max_radius + 1):
                li_tmp = []
                for node, dist in entry[1].items():
                    if dist == radius:
                        li_tmp.append(node)
                if li_tmp != []:
                    di_tmp[radius - 1] = li_tmp
            result.append(copy.copy(di_tmp))
            
    return result

separation_degrees = change_ds(spl, 1000, 10)
print(time.time() - start)

In [ ]:
''' 7.
    For NEWER version '''
def infected_percentage(nodes, infected) :
    count = 0
    for node in nodes :
        if node in infected :
            count += 1
    
    return count / len(nodes)

def infected_average_by_separation_degree(G, separation_degrees, infected) :
    node_count = {}
    percentage_sum = {}
    
    for node in range(0, len(separation_degrees)) :
        if node in infected :
            for degree in separation_degrees[node] :
                if degree not in percentage_sum.keys() :
                    percentage_sum[degree] = 0
                    node_count[degree] = 0
                percentage_sum[degree] += infected_percentage(separation_degrees[node][degree], infected)
                node_count[degree] += 1
        
    average_by_separation_degree = {}
    
    for separation_degree in percentage_sum :
        average_by_separation_degree[separation_degree] = percentage_sum[separation_degree] / node_count[separation_degree]
            
    return average_by_separation_degree


In [ ]:
''' 4. '''
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics.SIModel as si

# Model selection
model = si.SIModel(G)

# Model Configuration
cfg = mc.Configuration()
cfg.add_model_parameter('beta', 0.01) # prob of infecteded
cfg.add_model_parameter("percentage_infected", 0.5) # initially infected  %
model.set_initial_status(cfg)

# Simulation execution
iterations = model.iteration_bunch(60)

In [ ]:
''' 5.
    For OLDER version '''
infected = []
count = 1
for it in iterations :
    print('Iteration ', count)
    count += 1
    for node, i in it['status'].items() :
        if i == 1:
            infected.append(node)
            
    print('Infected count', len(infected))
    #kawai_draw(G)    
    print(infected_average_by_radius(G, radius_split, 10, infected))

In [ ]:
''' 8.
    For NEWER version '''
show_degrees = 10
infected = []
for it in range(0, len(iterations)) :
    print('Iteration', it+1)
    for node, i in iterations[it]['status'].items() :
        if i == 1:
            infected.append(node)
            
    print('Infected count', len(infected))
    #kawai_draw(G)
    percentages = infected_average_by_separation_degree(G, separation_degrees, infected)
    
    print('Separations:')
    to_show = []
    for d in range(0, show_degrees) :
        if d in percentages :
            to_show.append(percentages[d])
        else :
            break
    
    print(to_show)